In [13]:
import papermill as pm
import pickle5 as pickle
pic_='F'
tracking_uri_='http://34.128.104.38:5000'

In [14]:
import mlflow
from mlflow.tracking import MlflowClient
model_tuning={}

In [15]:
client = MlflowClient()
experiment = client.get_experiment_by_name("Hyper Parameter Tuning")
if experiment!=None:
    exp_id=experiment.experiment_id
    data_=mlflow.search_runs([exp_id], order_by=["metrics.f1_micro DESC","metrics.logloss ASC","metrics.roc_auc_score DESC"])

In [16]:
best_cat=data_.loc[data_['params.algorithm']=='catboost'].iloc[0,:]['run_id']
best_cat='runs:/'+best_cat+'/catboost_model'
best_cat=mlflow.catboost.load_model(best_cat)

best_lgb=data_.loc[data_['params.algorithm']=='lightgbm'].iloc[0,:]['run_id']
best_lgb='runs:/'+best_lgb+'/lgbm_model'
best_lgb=mlflow.lightgbm.load_model(best_lgb)

best_rf=data_.loc[data_['params.algorithm']=='randomforest'].iloc[0,:]['run_id']
best_rf='runs:/'+best_rf+'/rf_model'
best_rf=mlflow.sklearn.load_model(best_rf)

In [17]:
model_tuning['catboost']=best_cat
model_tuning['lightgbm']=best_lgb
model_tuning['randomforest']=best_rf

with open('model/tuning/model_tuned.pickle', 'wb') as handle:
    pickle.dump(model_tuning, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [18]:
import os
from google.cloud import storage
client = storage.Client()
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'gcp_store.json'
bucket = client.get_bucket('bps-gcp-bucket')
file_name='MLST2023/model/tuning-model/'+pic_+'_tuning.pickle'
bucket.blob(file_name).upload_from_filename('model/tuning/model_tuned.pickle')